In [1]:
!/Users/thai/anaconda2/lib/python2.7/site-packages/shadow_useragent/getpk.sh

In [2]:
import time 
import shadow_useragent
import copy, sys, os, re, random
import glob
import threading
import bs4 as bs
import pandas as pd
import requests
from datetime import date
import queue
from fake_useragent import UserAgent
from urllib2 import urlopen, Request
from urllib2 import HTTPError
import warnings
warnings.filterwarnings("ignore")

In [3]:
sys.setrecursionlimit(10**6)

In [4]:
PROXIE_LIST = ['http://144.202.65.136:1441', 'http://144.202.65.136:1440',
               'http://144.202.65.136:1441', 'http://144.202.65.136:1442',
               'http://144.202.65.136:1443','http://144.202.65.136:1444']
new_cols = ['Category 1', 'Category 2', 'Category 3', 'Enrollment', 'Language', 'Description', 'Last Update', 'Date Scraped']

In [5]:
def scrape_html(html, date_scraped):
    soup = bs.BeautifulSoup(html, 'lxml')
    
    private = soup.find('div', attrs = {'class':'col-sm-4 col-sm-push-4 col-xs-8 col-xs-push-2'})
    private_2 = soup.find('i', attrs = {'class': 'udi udi-warning'})
    
    if (private is not None) or (private_2 is not None):
        print "private"
        none = ["None"] * len(new_cols)
        return dict(zip(new_cols,none))
    
    enrollment = soup.find('div', attrs = {'data-purpose' : 'enrollment'})
    last_update = soup.find('div', attrs = {'class': 'last-update-date'})
    language = soup.find('div', attrs = {'class': 'clp-lead__locale'})
    description = soup.find('div', attrs = {'class' : 'clp-lead__headline'})
    categories = [cat.text.strip("\n") for cat in soup.find_all('a', attrs = {'class': 'topic-menu__link'})]
    
    try:
        cat = [" ", " ", " "]
        for idx in range(len(categories)):
            cat[idx] = categories[idx]
        
        data = [cat[0], cat[1], cat[2], enrollment.text.strip(" \n").split(" ")[0], 
                language.text.strip("\n"), description.text.strip("\n"),
                last_update.text.strip("\n").split(" ")[-1], date_scraped]
        
        data_dict = dict(zip(new_cols,data))
        
        return data_dict
    except Exception:
        raise Exception()

In [6]:
def scrape_single_url(url, proxies, user_agent, date_scraped):
    ua = shadow_useragent.ShadowUserAgent()
    agent = ua.percent(0.05)
    headers = {'user-agent': user_agent}
        
    proxies = { 'http' : random.choice(PROXIE_LIST) }
    
    try:
        ua = shadow_useragent.ShadowUserAgent()
        agent = ua.percent(0.05)
        headers = {'user-agent': user_agent}
        
        proxies = { 'http' : random.                                                                    (PROXIE_LIST) }
        
        req = requests.get(url=url, proxies=proxies, headers=headers, timeout=6)
        req.encoding = 'ISO-8859-1'
        html = req.text
        if (req.url != url):
            print "Course Removed"
            none = ["None"] * len(new_cols)
            return dict(zip(new_cols,none))
        
        if (req.status_code == 200): 
            data = scrape_html(html, date_scraped)
            return data
        elif req.status_code == 403:
            print req.status_code
            time.sleep(1800)
            return scrape_single_url(url, proxies, agent, date_scraped)
        else:
            print req.status_code
            time.sleep(3)
            return scrape_single_url(url, proxies, agent, date_scraped)
    except Exception as e:
        print e
        time.sleep(2)
        return scrape_single_url(url, proxies, agent, date_scraped)

In [7]:
def pull_info(csv_in, proxies, user_agent, date_scraped):
    out_file = os.path.basename(csv_in)
    df = pd.read_csv(csv_in)
    if 'Category 1' in df:
        df = df.drop(['Category 1','Category 2', 'Category 3'], axis=1)

    urls = list(df['URL'])
    
    new_frame = pd.DataFrame(columns=new_cols)
        
    for idx in range(len(df.index)):
        url = 'https://' + urls[idx]
        print str(idx) + "/" + str(len(df)) + ": " + url
        time.sleep(2)
        if (idx+1 % 300 == 0):
            time.sleep(90)
        data = scrape_single_url(url, proxies, user_agent, date_scraped)
        new_frame = new_frame.append(data, ignore_index=True)
    df = pd.concat([df, new_frame], axis=1)
    df.to_csv('~/Desktop/Kris/final/'+out_file, index = None, header=True,encoding = 'utf-8')
    
    print csv_in + " :DONE"

    return df

In [8]:
def task(csv, proxies):
    ua = shadow_useragent.ShadowUserAgent()
    agent = ua.percent(0.05)
    today = date.today()
    date_string = today.strftime("%m/%d/%y")
    df = pull_info(csv, proxies, agent, date_string)

In [9]:
done = [os.path.split(s)[1] for s in glob.glob(os.getcwd() + "/final/*.csv")]
all_files = glob.glob('/Users/thai/Desktop/Kris/out'+ "/*.csv")
todos = [os.path.split(f)[1] for f in all_files if os.path.split(f)[1] not in done]

In [10]:
def worker():
    while True:
        item = q.get()
        if item is None:
            break
        proxies = {'http': random.choice(PROXIE_LIST)}
        task(item,proxies)
        time.sleep(30)

In [11]:
q = queue.Queue()
for item in todos:
    q.put('/Users/thai/Desktop/Kris/out/' + item)
q.put(None)

In [ ]:
worker()

0/201: https://www.udemy.com/course/3-minute-french-full-course-1/
1/201: https://www.udemy.com/course/3-minute-french-course-2/
2/201: https://www.udemy.com/course/intermediate-french-master-the-past-tenses/
3/201: https://www.udemy.com/course/frenchpronunciation/
4/201: https://www.udemy.com/course/francais-intensif-intensive-french-course-for-beginners/
5/201: https://www.udemy.com/course/3-minute-french-course-3/
6/201: https://www.udemy.com/course/french-subjunctive/
7/201: https://www.udemy.com/course/french-for-beginner-course-level-1/
8/201: https://www.udemy.com/course/3-minute-french-course-4/
9/201: https://www.udemy.com/course/francais-intensif-intensive-french-intermediate-level/
10/201: https://www.udemy.com/course/french-for-you-course-for-beginners-and-false-beginners/
11/201: https://www.udemy.com/course/conversational-french-made-easy/
12/201: https://www.udemy.com/course/3-minute-french-course-5/
13/201: https://www.udemy.com/course/3-minute-french-course-6/
14/201: 

110/201: https://www.udemy.com/course/turkler-icin-sfrdan-ileri-duzey-franszca-kursu-2-edition/
111/201: https://www.udemy.com/course/il-francese-per-te/
112/201: https://www.udemy.com/course/curso-de-frances-conjugaciones-en-frances/
113/201: https://www.udemy.com/course/curso-de-frances-basico-aprenda-com-aulas-de-frances-online-a/
114/201: https://www.udemy.com/course/the-complete-french-grammar-course-from-delf-a1-to-dalf-c1/
115/201: https://www.udemy.com/course/french-course-advanced-dalf-c1-cefrl-official-certificate/
116/201: https://www.udemy.com/course/french-course-independant-delf-b2-cefrl-official-certificate/
117/201: https://www.udemy.com/course/curso-basico-frances/
118/201: https://www.udemy.com/course/lessentieldelorthographe/
119/201: https://www.udemy.com/course/french-course-proficient-dalf-c2-cefrl-official-certificate/
120/201: https://www.udemy.com/course/curso-basico-de-frances-curso-completo-para-iniciantes/
121/201: https://www.udemy.com/course/french-course-

15/50: https://www.udemy.com/course/how-to-draw-the-human-figure-10-hours/
16/50: https://www.udemy.com/course/figure-drawing-for-beginners/
17/50: https://www.udemy.com/course/you-can-sketch-people/
18/50: https://www.udemy.com/course/drawing-the-human-figure/
19/50: https://www.udemy.com/course/drawing-human-actions-to-master-figure-drawing-and-painting/
20/50: https://www.udemy.com/course/figure-drawing-the-next-level/
21/50: https://www.udemy.com/course/urban-sketchers-easy-tips-to-help-you-put-people-in-scenes/
22/50: https://www.udemy.com/course/figmaflutter/
23/50: https://www.udemy.com/course/fashion-figure-sketches-quick-and-easy/
24/50: https://www.udemy.com/course/figure-drawing-fundamentals/
25/50: https://www.udemy.com/course/complete-portrait-and-figure-drawing-step-by-step/
26/50: https://www.udemy.com/course/easy-steps-to-figure-drawing-anatomy-of-male-female-body/
27/50: https://www.udemy.com/course/learn-clear-easy-ways-to-draw-draw-basic-human-figures/
28/50: https:/

1/269: https://www.udemy.com/course/blender-environments/
2/269: https://www.udemy.com/course/blender-28-the-complete-guide-from-beginner-to-pro/
3/269: https://www.udemy.com/course/blender3dmodelingguide/
4/269: https://www.udemy.com/course/blender-281-substance-painter-sci-fi-asset-creation/
5/269: https://www.udemy.com/course/learn-blender-3d-modeling-for-unity-video-game-development/
6/269: https://www.udemy.com/course/autocad3d/
7/269: https://www.udemy.com/course/3d-modeling-in-blender-2-8-for-unity-video-game-developers/
8/269: https://www.udemy.com/course/next-gene-assets-for-videogames/
9/269: https://www.udemy.com/course/lowpolyblender2-8/
10/269: https://www.udemy.com/course/siemens-nx-unigraphics-ug-basic-to-advance-3d-modelling/
11/269: https://www.udemy.com/course/create-a-character-volume1/
12/269: https://www.udemy.com/course/rhino-3d-v6-or-v5-level-2-ship-surfacing/
13/269: https://www.udemy.com/course/blender-2-8-creating-your-first-3d-game-model/
14/269: https://www.

104/269: https://www.udemy.com/course/learn-blender-3d-the-complete-modelling-course/
105/269: https://www.udemy.com/course/creating-the-soul-reaver-sword-in-zbrush/
106/269: https://www.udemy.com/course/3dmotive-learn-to-model-and-texture-3d-assets-for-mobile-games/
107/269: https://www.udemy.com/course/maya-3d-modelling-and-rendering-iconic-designs-extended/
108/269: https://www.udemy.com/course/rhino-grasshopper-architectural-canopy-with-curve-attractor/
109/269: https://www.udemy.com/course/creating-and-monetizing-3d-models-for-beginners/
110/269: https://www.udemy.com/course/master-parametric-3d-modeling-for-3d-printing-with-inventor/
111/269: https://www.udemy.com/course/beginners-3d-modelling-14-low-poly-buildings-for-games/
112/269: https://www.udemy.com/course/solidworks2019/
113/269: https://www.udemy.com/course/learn-photoshop-3d-and-create-realistic-photo-manipulations/
114/269: https://www.udemy.com/course/modelling-a-stylized-tree-in-maya/
115/269: https://www.udemy.com/c

...

81/348: https://www.udemy.com/course/modelphotography/
82/348: https://www.udemy.com/course/raw-to-perfect-photo/
83/348: https://www.udemy.com/course/earn-from-photography-complete-guideline/
84/348: https://www.udemy.com/course/3d-photography/
85/348: https://www.udemy.com/course/understanding_exposure_using_manual_mode_dslr/
86/348: https://www.udemy.com/course/shoot-stunning-portraits-of-your-kids/
87/348: https://www.udemy.com/course/photography-for-beginners-master-camera-settings/
88/348: https://www.udemy.com/course/product-photography-bts-lesson-from-a-pro/
89/348: https://www.udemy.com/course/how-to-manage-a-profitable-photography-studio/
90/348: https://www.udemy.com/course/become-a-better-photographer/
91/348: https://www.udemy.com/course/how-to-take-great-photos-made-simple/
92/348: https://www.udemy.com/course/accredited-photography-diploma/
93/348: https://www.udemy.com/course/wedding-photography-how-to-attract-luxury-weddings/
94/348: https://www.udemy.com/course/create

192/348: https://www.udemy.com/course/manual-photography-use-your-camera-wisely/
193/348: https://www.udemy.com/course/how-to-create-a-10000-dollars-photography-website-for-less-than-100/
194/348: https://www.udemy.com/course/freelensing-tutorial/
195/348: https://www.udemy.com/course/the-ultimate-guide-to-digital-photography-for-beginners/
196/348: https://www.udemy.com/course/the-beginners-guide-to-photography/
197/348: https://www.udemy.com/course/create-a-gorgeous-photography-site-with-smugmug-in-90-mins/
198/348: https://www.udemy.com/course/secrets-of-magazine-model-photography/
199/348: https://www.udemy.com/course/light-painting-in-360-with-the-ricoh-theta-s/
200/348: https://www.udemy.com/course/exposure-best-practices/
201/348: https://www.udemy.com/course/learn-photography-by-vishal-diwan/
202/348: https://www.udemy.com/course/photography-learn-photography-from-pro-take-better-photos/
203/348: https://www.udemy.com/course/ultimate-landscape-photography-guide/
204/348: https:

305/348: https://www.udemy.com/course/livephoto/
306/348: https://www.udemy.com/course/historia-de-la-fotografia-i/
307/348: https://www.udemy.com/course/etkileyici-fotograf-duzenleme-teknikleri-ii/
308/348: https://www.udemy.com/course/flash-speedlight-canon-para-eventos/
309/348: https://www.udemy.com/course/photoshop-correzione-del-colore-di-venti-fotografie/
310/348: https://www.udemy.com/course/fotos-im-gemaeldelook/
311/348: https://www.udemy.com/course/kurs-capture-one-10-zarzadzanie-i-edycja-raw/
312/348: https://www.udemy.com/course/fotografia-basica-para-venta-en-internet-stock-photography/
313/348: https://www.udemy.com/course/photomasa/
314/348: https://www.udemy.com/course/vende-tus-fotografias-con-microstock/
315/348: https://www.udemy.com/course/shohin01/
316/348: https://www.udemy.com/course/curso-de-fotografia-profissional/
317/348: https://www.udemy.com/course/fotografien-richtig-und-erfolgreich-verkaufen/
318/348: https://www.udemy.com/course/fotografie-basis-cursus/

36/170: https://www.udemy.com/course/essential-ielts-vocabulary/
37/170: https://www.udemy.com/course/academic-ielts-writing-task-1/
38/170: https://www.udemy.com/course/comprehensive-english-grammar/
39/170: https://www.udemy.com/course/toefl-ielts-vocabulary-practice-test-all-you-need-to-pass/
40/170: https://www.udemy.com/course/ielts-band-7-academic-writing-preparation-course/
41/170: https://www.udemy.com/course/ielts-help/
403
42/170: https://www.udemy.com/course/ielts-writing-task-2-intensive-course/
43/170: https://www.udemy.com/course/ielts-speaking-7-plus/
44/170: https://www.udemy.com/course/unlock-the-ielts-score-band-7-readinglistening/
45/170: https://www.udemy.com/course/ielts-task-1-bootcamp-academic-module/
46/170: https://www.udemy.com/course/understanding-ielts-exam-the-basics/
47/170: https://www.udemy.com/course/score-high-ielts-speaking/
48/170: https://www.udemy.com/course/ielts-9-speaking-mastery/
49/170: https://www.udemy.com/course/ielts-listening-tips-tricks-

146/170: https://www.udemy.com/course/ielts-general-training-writing-task-2/
147/170: https://www.udemy.com/course/bgiconsultancy/
148/170: https://www.udemy.com/course/father-ielts-speaking-test-preparation-get-a-band-7/
149/170: https://www.udemy.com/course/ielts-writing-with-nas-cee/
403
150/170: https://www.udemy.com/course/ielts-speaking-boost-enhance-speaking-skills-from-6-to-7/
151/170: https://www.udemy.com/course/ielts-speaking-part-two-20181/
152/170: https://www.udemy.com/course/father-ielts-test-preparation/
153/170: https://www.udemy.com/course/ielts-writing-part-two-opinion-from-question-to-band-7-9/
154/170: https://www.udemy.com/course/ielts-masterclass-the-fast-track-to-speaking-like-a-native/
155/170: https://www.udemy.com/course/ielts-daily-bites-free-course/
156/170: https://www.udemy.com/course/how-to-boost-your-ielts-speaking-band-score/
157/170: https://www.udemy.com/course/ielts-academic-reading-listening-achieve-band-9/
158/170: https://www.udemy.com/course/iel

In [ ]:
# THREADS = 3

In [ ]:
# threads = []
# for i in range(THREADS):
#     time.sleep(1)
#     t = threading.Thread(target=worker)
#     t.start()
#     threads.append(t)

In [ ]:
# for t in threads:
#     t.join()